In [ ]:
from collections import OrderedDict
from pymatgen import MPRester
import requests

In [ ]:
material_ids = OrderedDict([
    ('diamond', 'mp-66'),  # C
    ('gaas', 'mp-2534'),  # GaAs
    ('rocksalt', 'mp-22862'),  # NaCl
    ('cubic_perovskite', 'mp-2998'),  # BaTiO3
    ('tetragonal_perovskite', 'mp-5986'),  # BaTiO3
    ('trigonal_perovskite', 'mp-5020'),  # BaTiO3
    ('orthorhombic_perovskite', 'mp-5777'),  # BaTiO3
    ('zincblende', 'mp-10695'),  # ZnS
    ('wurtzite', 'mp-10281'),  # ZnS
    ('fcc', 'mp-23'),  # Ni
    ('big_fcc', 'mp-76'),  # Sr
    ('bcc', 'mp-13'),  # Fe
    ('big_bcc', 'mp-70'),  # Rb
    ('hcp', 'mp-153'),  # Mg
    ('trigonal', 'mp-782'),  # Te2Pd
    ('tetragonal', 'mp-742'),  # Ti2Cu
    ('monoclinic', 'mp-684'),  # BaS2
    ('triclinic', 'mp-9122'),  # CaP3
    ('orthorhombic', 'mp-872')  # BaSn
])

In [ ]:
with MPRester('0WqdPfXxloze6T9N') as mpr:
    structures = OrderedDict([(name, mpr.get_structure_by_material_id(id)) for name, id in material_ids.iteritems()])

In [ ]:
endpoints = {'distances': ['matminer', 'pymatgen', 'soap'],
             'comparisons': ['matminer', 'pymatgen', 'soap'],
             'fingerprints': ['matminer', 'stidy', 'soap']}
for endpoint, functions in endpoints.items():
    for function in functions:
        print(endpoint, function)
        r = requests.get('http://127.0.0.1:5000/v1/{}/{}/'.format(endpoint, function), params=payload)
        results = json.loads(r.text)
        print(results)